# Лабораторная работа №1

Импортируйте библиотеку pandas, а затем загрузите данные:

In [15]:
import pandas as pd
taxi = pd.read_csv('./data/taxi_nyc.csv', parse_dates=['pickup_dt'])

In [16]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


Проверьте, сколько всего строк и столбцов имеется в датасете:

In [17]:
taxi.shape

(29101, 14)

Посмотрим на типы колонок:

In [18]:
taxi.dtypes

pickup_dt       datetime64[ns]
pickup_month            object
borough                 object
pickups                  int64
hday                    object
spd                    float64
vsb                    float64
temp                   float64
dewp                   float64
slp                    float64
pcp 01                 float64
pcp 06                 float64
pcp 24                 float64
sd                     float64
dtype: object

В названиях столбцов pcp 01, pcp 06, pcp 24 заменить пробелы:

In [19]:
taxi = taxi.rename(columns={'pcp 01': 'pcp_01', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'})
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp_01,pcp_06,pcp_24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


Сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan):

In [20]:
len(taxi.query('borough == "Manhattan"'))

4343

Выяснить, из какого района было совершено наибольшее количество поездок за весь период:

In [21]:
taxi.groupby(['borough'], as_index=False) \
    .agg({'pickups': 'sum'}) \
    .sort_values('pickups', ascending=False)


,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


Сгруппируйте данные по району (borough) и укажите, из какого пункта было совершено больше всего поездок

In [22]:
min_pickups = taxi.groupby('borough').agg({'pickups' : 'sum'})
min_pickups.idxmin()

pickups    EWR
dtype: object

Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). 
Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни:

In [23]:
taxi_hday = taxi.groupby(['borough', 'hday'], as_index=False) \
    .agg({'pickups': 'mean'})

hday = taxi_hday.query('hday == "Y"') \
    .rename(columns={'pickups': 'pickups_hday'})
not_hday = taxi_hday.query('hday == "N"') \
    .rename(columns={'pickups': 'pickups_not_hday'})

pd.merge(not_hday[['borough', 'pickups_not_hday']], hday[['borough', 'pickups_hday']], on='borough')

,borough,pickups_not_hday,pickups_hday
0,Bronx,50.771073,48.065868
1,Brooklyn,534.727969,527.011976
2,EWR,0.023467,0.041916
3,Manhattan,2401.302921,2035.928144
4,Queens,308.899904,320.730539
5,Staten Island,1.606082,1.497006


Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию:

In [24]:
taxi.groupby(['borough', 'pickup_month'], as_index=False) \
    .agg({'pickups' : 'sum'}) \
    .sort_values('pickups', ascending=False)

,borough,pickup_month,pickups
21,Manhattan,Jun,1995388
23,Manhattan,May,1888800
19,Manhattan,Feb,1718571
22,Manhattan,Mar,1661261
18,Manhattan,Apr,1648278
20,Manhattan,Jan,1455543
9,Brooklyn,Jun,482466
11,Brooklyn,May,476087
6,Brooklyn,Apr,378095
10,Brooklyn,Mar,346726


Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F, и возвращает значения, переведенные в градусы Цельсия:

In [25]:
def temp_to_celcius(temp): 
    return  (temp - 32) * 5.0/ 9.0

taxi['celsius_temp'] = temp_to_celcius(taxi['temp'])

taxi['temp'][:5]
taxi['celsius_temp'][:5]

0   -1.111111
1   -1.111111
2   -1.111111
3   -1.111111
4   -1.111111
Name: celsius_temp, dtype: float64